<h2><b>Analysis of the impact of the Ukrainian war on YouTube channels of some Russian bloggers about politic</b></h2>

<h3>Project description</h3>

There were a lof of changes for many russians after 24th February 2022, when the unfair dicision about the invasion in Ukraine was taken by the malicious russian president. Answers on lots of questions were needed, like "how to live further?", "why it's happening?", "is there any valid reason?" and so forth. To get the answers the majority of russian people were getting an information from independent medias using platforms such as YouTube, Telegram and so on.

This project was made to find (if they were) and analyse changes in some popular YouTube channels of bloggers, who talks about russian politics.

Next questions shoud be answered:
1. How activity of the bloggers was changed after 24th Februry 2022?
2. How popularity of the channels was changed after 24th Februry 2022?
3. Which is the most popular word accorting to tags?


The most channels, which were choosen for the current project, are so-called liberal, because almost all russian propaganda channels have been blocked by YouTube administration. As the result, the initial data could be lightly biased. But, anyway, the results of the project, hopefully, will shows how independed media are important and in demand.

For the project next channels were choosen to analyse:
1. https://www.youtube.com/@Ekaterina_Schulmann
2. https://www.youtube.com/@varlamov
3. https://www.youtube.com/@Max_Katz
4. https://www.youtube.com/@besogontv (pro-Kremlin)

<h4>Used libraries</h4>

All custom function are store separately in the file yout_an_func.py

In [1]:
# for work with YouTube API
from googleapiclient.discovery import build

# for data processing
import pandas as pd
import sqlite3 as sq
from dateutil import parser
import isodate
import openpyxl
from itertools import chain

# for data visualization
import panel as pn
pn.extension()

import hvplot.pandas
from wordcloud import WordCloud

#  custom functions
import yout_an_func as yaf 

<h3>Collecting the data</h3>

<h4>Stage description</h4>

Since necessary data is on YouTube the best way to get it is to use YouTube API. For this purpose next steps are required:
- manually find the YouTube channels ID on the platform;
- find owned API key on console.cloud.google.com; 
- explore an official documentation about YouTube API https://developers.google.com/.
<br><br>

After studying YouTube API documentation the next actions were decided:
1. using YouTube API resource "channels" and method "list" general data about channels will be download and put into DataFrame (channel_info);
2. using channels playlist ID from the recieved information about channels and YouTube API resourse "playlistItems" the array (python list) with all videos IDs will be created (video_ids);
3. using YouTube API resource "videos" and the abovementioned array with video IDs each video data will be extract and stored in another DataFrame (video_df).

Created tables will have all data wich is required for further analysis and getting the results.

In [2]:
# Екатерина Шульман (id UCL1rJ0ROIw9V1qFeIN0ZTZQ)
# Илья Варламов  (id UC101o-vQ2iOj9vr00JUlyKw)
# Максим Кац (id UCUGfDbfRIx51kJGGHIFo8Rw)
# Бесогон ТВ (id UCiwIy5q59aD-Nph5f-Zvp2A)
channel_ids = ['UCiwIy5q59aD-Nph5f-Zvp2A', 'UCL1rJ0ROIw9V1qFeIN0ZTZQ',
'UC101o-vQ2iOj9vr00JUlyKw', 'UCUGfDbfRIx51kJGGHIFo8Rw']  

# get owned API key from console.cloud.google.com
api_key = ""

# Get credentials and create an API client
youtube = build("youtube", "v3", developerKey=api_key)

<h4>Getting information from YouTube using custom module "yout_an_func.py" </h4>

In [3]:
# step 1: creating DataFrame with channels data
channel_info = yaf.get_channel_info(youtube, channel_ids)

# step 2: creating the list with all videos ID
all_video_ids = yaf.full_list_of_videos(youtube, channel_info)

# step 3: creating DataFrame with videos data
video_df = yaf.get_video_details(youtube, all_video_ids)

# naming created tables for further convenience
channel_info.name = "Channels"
video_df.name = "Videos"

# checking the sizes of created tables, they must be plausible
print(f'''{channel_info.name}: {channel_info.shape}, 
{video_df.name}: {video_df.shape}''')

Channels: (4, 9), 
Videos: (3759, 10)


<h3>Exploring and pre-processing the data</h3>

<h4>Stage description</h4>

Since the data was collected straight from YouTube, so-called second-party data, inherently it should be homogeneous and shouldn't have duplicates. However, there is still work to be done in the pre-processing stage, such as:
1. identification and analysis of missing values, making a decision on the need for any operations with them;
2. data type checking and correction;
3. data normalization if needed.

In [4]:
# making copies of created DF just to be on the safe side
c_channel_info = channel_info.copy()
c_video_df = video_df.copy()

<h4>Missing data</h4>

During this step it's necessary to check which columns in the created tables have NULL values and make sure that they do not negatively  affect on the final result of the project.

Let's find columns with null values:

In [5]:
yaf.miss_data_cnames(channel_info, video_df)

There is not missing date in table "Channels"
Table "Videos" has 618 null values in column "tags"
Table "Videos" has 5 null values in column "commentCount"


The above result expected shows that there is no missing of crusial data. But for further convenience, in the column "commentCount" we'll fill Null values as 0. 

In [6]:
video_df['commentCount'] = video_df['commentCount'].fillna(0)

<h4>Data type</h4>

For further analysis it's necessary that each column has a relevant type of data. Columns with quantitative data should have numeric type of data.

In [7]:
# checking datatypes in tables
print(channel_info.dtypes)
video_df.dtypes

channelId             object
channelName           object
customURL             object
channelDescribtion    object
subscribers           object
views                 object
totalVideos           object
playlistId            object
topicCategories       object
dtype: object


video_id        object
channelId       object
title           object
description     object
tags            object
publishedAt     object
viewCount       object
likeCount       object
commentCount    object
duration        object
dtype: object

Numeric columns.

As we see all columns have one data type, which is not correct. There are several columns which obviously should be numeric. Now let's fix it.

In [8]:
# defining numeric columns for Channels table and updating data type
numeric_cols = ['subscribers', 'views', 'totalVideos']
channel_info[numeric_cols] = channel_info[numeric_cols].apply(pd.to_numeric)

# defining numeric columns for Videos table and updating data type
numeric_cols = ['viewCount', 'likeCount', 'commentCount']
video_df[numeric_cols] = video_df[numeric_cols].astype("int64")

Datetime data.

There are two columns with values about date and/or time in "Videos" table. Let's convert them to a proper format.

In [9]:
video_df['publishedAt'] = video_df['publishedAt'].apply(lambda x: parser.parse(x)) # parsing data
video_df['publishedAt'] = pd.to_datetime(video_df['publishedAt'].dt.tz_localize(None), \
    format="%m/%d/%Y, %H:%M:%S") # removing time zone and changing data format

#converting duration column from strange YouTube format into seconds, but it'll store in float64 type data
video_df['duration'] = video_df['duration'].apply(lambda x: isodate.parse_duration(x).total_seconds()) 

<h4>Data normalization</h4>

During this stage data will be checked more carefully and it'll be modifyed.

At first let's take a look on the table "Channels"

In [10]:
channel_info.head(2)

,channelId,channelName,customURL,channelDescribtion,subscribers,views,totalVideos,playlistId,topicCategories
0,UCiwIy5q59aD-Nph5f-Zvp2A,БесогонTV,@besogontv,Официальный канал авторской программы Никиты М...,1490000,132274452,252,UUiwIy5q59aD-Nph5f-Zvp2A,"[https://en.wikipedia.org/wiki/Society, https:..."
1,UCL1rJ0ROIw9V1qFeIN0ZTZQ,Екатерина Шульман,@ekaterina_schulmann,Официальный канал Екатерины Шульман\n\nДАННЫЙ ...,1070000,144715177,904,UUL1rJ0ROIw9V1qFeIN0ZTZQ,"[https://en.wikipedia.org/wiki/Politics, https..."


Everything seems fine except column "topicCategories". There are links in the column and they are in brackets, what looks like list.

Also it'll be usefull to and a column "type" where the channels will be marked as liberal and propaganda.

In [11]:
# remove link to wiki and list format (db won't get it)
channel_info['topicCategories'] = channel_info['topicCategories'].apply(lambda x: \
    str(x).lstrip("[").rstrip("]").replace("https://en.wikipedia.org/wiki/", "").replace("'", ""))

# add type column. i'll define variable with the list of propaganda channels
pr_channels = ['UCiwIy5q59aD-Nph5f-Zvp2A']
# def normalise_row(row, pr_channels):
#     if row['channelId'] in pr_channels:
#         return 'propaganda'
#     else:
#         return 'liberal'
channel_info['type'] = channel_info.apply(lambda row : yaf.normalise_row(row, pr_channels), axis=1) 

Now let's take a look on the table "Videos".

In [12]:
video_df.head(2)

,video_id,channelId,title,description,tags,publishedAt,viewCount,likeCount,commentCount,duration
0,JeaKZlqhTz0,UCiwIy5q59aD-Nph5f-Zvp2A,БесогонТВ «Жёны маркиза Карабаса»,В новом выпуске авторской программы «БесогонТВ...,"[Михалков, Никита Михалков, Бесогон, БесогонТВ...",2023-02-10 09:20:00,810463,117970,20808,3780.0
1,kMwLWpSWaZ0,UCiwIy5q59aD-Nph5f-Zvp2A,Анонс нового выпуска БесогонТВ «Жёны маркиза К...,В новом выпуске авторской программы «БесогонТВ...,"[Михалков, Никита Михалков, анонс нового выпус...",2023-02-08 07:30:06,316781,30427,3844,36.0


It's necessary to remove brackets in coluumn "tags" and it'll be usefull to add column "pushblishDayName" with the weekday when a video was published.

In [13]:
video_df['pushblishDayName'] = video_df['publishedAt'].apply(lambda x: x.strftime("%A"))  # adding weekdays

video_df['tags'] = video_df['tags'].apply(lambda x: str(x).lstrip("[").rstrip("]").replace("'", "")) # making some changes to tags column

Creating additional table

One more thing which needed to be done is to create a separate table with row for each tag.

In [14]:
tag_df = pd.DataFrame(columns=['video_id', 'tag'])
for i, row in video_df.iterrows():
    if row['tags'] is not None:
        tag_l = row['tags'].split(', ')
        for tag in tag_l:
            if tag is not None and tag != 'None':  # actually I don't know why, but is not None remove only part of all None tags
                tmp_df = pd.DataFrame({'video_id': [row['video_id']], 'tag': [tag.lower()]})
                tag_df = pd.concat([tag_df, tmp_df], ignore_index = True)

<h3>Saving data in database</h3>

All tables will be saved in database before analysis.

In [15]:
# create db and connection
db_name = 'youtube_ch.db'
conn = sq.connect(db_name)
cursor_name = conn.cursor()

# create and insert channel_info table
table_name = 'channels'
cursor_name.execute(f'''CREATE TABLE IF NOT EXISTS {table_name}(
    channelId VARCHAR(50) PRIMARY KEY,
    channelName VARCHAR(50) NOT NULL,
    customURL VARCHAR(50),
    channelDescribtion VARCHAR(150),
    subscribers INT,
    views INT,
    totalVideos INT,
    playlistId VARCHAR(150),
    topicCategories TEXT,
    type VARCHAR(30)
    );''')
channel_info.to_sql(table_name,conn,if_exists='append', dtype={
    'channelId': 'VARCHAR(50)',
    'channelName': 'VARCHAR(50)',
    'customURL': 'VARCHAR(50)',
    'channelDescribtion': 'VARCHAR(150)',
    'subscribers': 'INT',
    'views': 'INT',
    'totalVideos': 'INT',
    'playlistId': 'VARCHAR(150)',
    'topicCategories': 'TEXT',
    'type': 'VARCHAR(30)'},index=False)
conn.commit()

# create and insert video_df table
table_name = 'videos'
cursor_name.execute(f'''CREATE TABLE IF NOT EXISTS {table_name}(
    video_id VARCHAR(50) PRIMARY KEY,
    channelId VARCHAR(50),
    title VARCHAR(300) NOT NULL,
    description TEXT,
    tags TEXT,
    publishedAt DATETIME,
    viewCount INT,
    likeCount INT,
    commentCount INT,
    duration INT,
    pushblishDayName VARCHAR(50),
    FOREIGN KEY(channelId) REFERENCES channels(channelId)
    );''')
video_df.to_sql(table_name,conn,if_exists='append', dtype={
    'video_id': 'VARCHAR(50)',
    'channelId': 'VARCHAR(50)',
    'title': 'VARCHAR(300)',
    'description': 'TEXT',
    'tags': 'TEXT',
    'publishedAt': 'DATETIME',
    'viewCount': 'INT',
    'likeCount': 'INT',
    'commentCount': 'INT',
    'duration': 'INT',
    'pushblishDayName': 'VARCHAR(50)'},index=False)
conn.commit()

# create and insert tags table
table_name = 'tags'
cursor_name.execute(f'''CREATE TABLE IF NOT EXISTS {table_name}(
    video_id VARCHAR(50),
    tag VARCHAR(100),
    FOREIGN KEY(video_id) REFERENCES videos(video_id)
    ON DELETE CASCADE
    );''')
tag_df.to_sql(table_name,conn,if_exists='append', dtype={
    'video_id': 'VARCHAR(50)',
    'tag': 'VARCHAR(100)'},index=False)
conn.commit()

conn.close()

<h3>Analyzing the data</h3>

For analyzing and visualization data the Panel library will be used. It allows to create widgets and dynamic charts.
To achieve abomentioned goals next plots will be made:
1. Bar chart to analyse current channel's features, such as quantity of subscriders, videos and views;
2. Wordcloud with most popular tags for choosen period
3. 

In [16]:
# this should be run if data already is stored in db and there is no reasons to collect it again from YouTube
try:
    channel_info.shape()
    video_df.shape()
    tag_df.shape
except:
    db_name = 'youtube_ch.db'
    conn = sq.connect(db_name)
    channel_info = pd.read_sql('select * from channels', conn)
    video_df = pd.read_sql('''select v.video_id, v.channelId, v.title, v.description, v.tags, v.publishedAt, v.viewCount, 
v.likeCount, v.commentCount, v.duration, v.pushblishDayName, channelName from videos as v
JOIN channels as c ON v.channelId= c.channelId''', conn)
    tag_df = pd.read_sql('select * from tags', conn)
    tag_df = pd.read_sql(f'''select t.video_id, tag, publishedAt, channelName from tags as t
JOIN videos as v ON v.video_id = t.video_id
JOIN channels as c ON v.channelId= c.channelId''', conn)
    conn.close
    video_df['publishedAt'] = pd.to_datetime(video_df['publishedAt'])
    video_df['year'] = video_df['publishedAt'].dt.year
    video_df['likable'] = video_df['likeCount']/video_df['viewCount']
    tag_df['publishedAt'] = pd.to_datetime(tag_df['publishedAt'])
    tag_df['year'] = tag_df['publishedAt'].dt.year

In [17]:
# creating color map
colors = {
    'varlamov': '#1f77b4',
    'БесогонTV': '#ad2134',
    'Екатерина Шульман': '#a550a6',
    'Максим Кац': '#381c52'
}
def u_color(channels, colors=colors):
    result = []
    for i in channels:
        result.append(colors[i])
    return result
# creating widgets
# widget for choosing channels
w_choose_channel = pn.widgets.CheckButtonGroup(name='Channels', value=list(channel_info['channelName']), 
    options=list(channel_info['channelName']),orientation='vertical', button_type='primary')

# widget for choosing channel's fetures
w_channel_feat = pn.widgets.RadioButtonGroup(name='features', options=['subscribers', 'views', 'totalVideos'], 
    button_type='primary')

# widget for choosing year
w_year = pn.widgets.IntSlider(name='year', start=video_df['year'].min(), end=video_df['year'].max(), \
    value=2015, value_throttled=2015)
w_year.width=250

w_yearrange = pn.widgets.DateRangeSlider(name='Date range', start=video_df['publishedAt'].min(), \
    end=video_df['publishedAt'].max())

# widget for choosing video's features
w_video_feat = pn.widgets.Select(name='video features', options=['viewCount', 'likeCount', 'commentCount', \
    'likable', 'duration'], value='viewCount')

Current information about channels

Let's compare current channels main features using a bar chart.

In [18]:
@pn.depends(w_choose_channel.param.value)
@pn.depends(w_channel_feat.param.value)
def ch_info_plt(channels, feat):
    ch_plot = channel_info[channel_info['channelName'].isin(channels)].sort_values(by=feat, \
        ascending=False).hvplot.bar(x='channelName', y=feat, c='channelName', cmap=colors,\
            xlabel='', width=700, title="Current channel's information", height=280)
    return ch_plot

# c='channelName', cmap=colors,
#.sort_values(by=feat, ascending=False).\
ch_barplot = pn.bind(ch_info_plt, w_choose_channel, w_channel_feat)
pn.Column(w_choose_channel, w_channel_feat, ch_barplot)

BokehModel(combine_events=True, render_bundle={'docs_json': {'ab65244d-59bb-4f91-9fd3-b03e590eac75': {'defs': …

Column
    [0] CheckButtonGroup(button_type='primary', name='Channels', options=['БесогонTV', ...], orientation='vertical', value=['БесогонTV', ...])
    [1] RadioButtonGroup(button_type='primary', name='features', options=['subscribers', ...], value='subscribers')
    [2] ParamFunction(function, _pane=HoloViews)

Chanel dependencies

In [ ]:
@pn.depends(w_year.param.value_throttled)
def vid_scatter(year):
    vid_vl_scatt = video_df[video_df['year'] == year].hvplot.scatter(x='likeCount', y='viewCount', \
        by='channelName', alpha=0.7, width=350, legend=False, c=video_df['channelName'], cmap=colors)
    vid_vc_scatt = video_df[video_df['year'] == year].hvplot.scatter(x='commentCount', y='viewCount', \
        by='channelName', alpha=0.70, width=350, legend=False, c=video_df['channelName'], cmap=colors)
    vid_scatt_p = vid_vl_scatt + vid_vc_scatt
    return vid_scatt_p

vid_scatt = pn.bind(vid_scatter, w_year)
pn.Column(w_year, vid_scatt)

BokehModel(combine_events=True, render_bundle={'docs_json': {'e23a83e6-8140-47f0-ad47-e11857f19b84': {'defs': …

Column
    [0] IntSlider(end=2023, name='year', start=2009, value=2015, width=250)
    [1] ParamFunction(function, _pane=HoloViews)

Most popular video

Let's check the best videos within a given perion by choosen criteria.

In [20]:
@pn.depends(w_yearrange.param.value)
@pn.depends(w_choose_channel.param.value)
@pn.depends(w_video_feat.param.value)
def vid_barsr(yearr, channels, feat):
    vid_bch = video_df[(video_df['publishedAt']>=yearr[0])&(video_df['publishedAt']<=yearr[1])&\
        (video_df['channelName'].isin(channels))].sort_values(by=feat, ascending=False)[0:15].\
            sort_values(by=feat).hvplot.barh('title', feat, yaxis = None, c=video_df['channelName'], \
                cmap=colors, legend=False, height=260, width=320)
    return vid_bch

vid_barchrtr = pn.bind(vid_barsr, w_yearrange, w_choose_channel, w_video_feat)
pn.Column(w_choose_channel, w_video_feat, w_yearrange, vid_barchrtr)

BokehModel(combine_events=True, render_bundle={'docs_json': {'503cc147-8a90-45f2-94bc-a8708c802e60': {'defs': …

Column
    [0] CheckButtonGroup(button_type='primary', name='Channels', options=['БесогонTV', ...], orientation='vertical', value=['БесогонTV', ...])
    [1] Select(name='video features', options=['viewCount', ...], value='viewCount')
    [2] DateRangeSlider(end=Timestamp('2023-02-15 1..., name='Date range', start=Timestamp('2009-03-18 2..., value=(Timestamp('2009-03-18 21:..., value_end=Timestamp('2023-02-15 1..., value_start=Timestamp('2009-03-18 2...)
    [3] ParamFunction(function, _pane=HoloViews)

Number of videos by year

In [23]:
vid_quant_df = pd.DataFrame(video_df[['channelName', 'year']].groupby('year').value_counts())
new_df = vid_quant_df.reset_index()
new_df.columns = ['year', 'channelName', 'counts']

new_df.hvplot.line(x='year', y='counts', by='channelName')

:NdOverlay   [channelName]
   :Curve   [year]   (counts)

Change in average values ​​of video's indicators by years

In [24]:
@pn.depends(w_choose_channel.param.value)
@pn.depends(w_video_feat.param.value)
def vid_lines(channels, feat):
    vid_linech = video_df[video_df['channelName'].isin(channels)][['channelName', 'year', feat]].\
        groupby(['year', 'channelName']).mean(feat).hvplot.line(by='channelName', legend=False, height=260, \
            color=u_color(channels), width=380)
    return vid_linech

vid_linechrt = pn.bind(vid_lines, w_choose_channel, w_video_feat)
pn.Column(w_choose_channel, w_video_feat,  vid_linechrt)

BokehModel(combine_events=True, render_bundle={'docs_json': {'2d5151f5-d3d8-4bc5-917f-159ad026a734': {'defs': …

Column
    [0] CheckButtonGroup(button_type='primary', name='Channels', options=['БесогонTV', ...], orientation='vertical', value=['БесогонTV', ...])
    [1] Select(name='video features', options=['viewCount', ...], value='viewCount')
    [2] ParamFunction(function, _pane=HoloViews)

In [26]:
vid_lplt = video_df[['channelName', 'year', 'viewCount']].groupby(['year', 'channelName']).mean('viewCount').hvplot.line(by='channelName',\
    legend=False, height=260)
vid_lplt


:NdOverlay   [channelName]
   :Curve   [year]   (viewCount)

Most popular tags

Before the wordcloud will be created it is necessary to check the most populat tags in each channel since it is common practice that bloggers add tags with their or channel's names.

In [27]:
# analyzing the most popular tags in ach channel
for i in list(channel_info['channelName']):
        print(tag_df[['channelName', 'tag']][tag_df['channelName'] == i].groupby('channelName').\
                value_counts()[0:15])

channelName  tag            
БесогонTV    михалков           157
             никита михалков    155
             бесогон            104
             бесогонтв           71
             россия              53
             украина             51
             противостояние      48
             крым                48
             майдан              47
             война               47
             политика            46
             власть              45
             прилепин            45
             экономика           44
             протест             44
dtype: int64
channelName        tag              
Екатерина Шульман  екатерина шульман    330
                   шульман              325
                   политология          176
                   россия               101
                   статус                99
                   эхо москвы            75
                   госдума               73
                   общество              67
                   навальный 

In [28]:
skip_tags = ['варламов', 'varlamov', 'илья варламов', 'varlamov.ru', 'варламов ютуб', 'варламов илья',\
    'канал варламова', 'ролики варламова', 'варламов ру', 'илья варламов блог', 'zyalt', 'илья варламов ютуб',\
    'благоустройство', 'илья варламов жж',  'михалков', 'никита михалков', 'бесогон', 'бесогонтв', 'кац',\
    'максим кац', 'макс кац', 'максим катс', 'макс катс', 'кац предлагает победить', 'екатерина шульман',
    'шульман', 'максим курников', 'бесогон tv', 'besogon', 'варламов москва', 'варламов и кац', 'варламов сша']

In [29]:
db_name = 'youtube_ch.db'
conn = sq.connect(db_name)
tag_df = pd.read_sql(f'''select t.video_id, tag, publishedAt, channelName from tags as t
JOIN videos as v ON v.video_id = t.video_id
JOIN channels as c ON v.channelId= c.channelId
WHERE tag NOT IN ({', '.join(["'" + x + "'" for x in skip_tags])})''', conn)
conn.close
tag_df['publishedAt'] = pd.to_datetime(tag_df['publishedAt'])
tag_df['year'] = tag_df['publishedAt'].dt.year
tag_df['yearMonth'] = tag_df['publishedAt'].dt.strftime('%Y-%m')

In [30]:
@pn.depends(w_yearrange.param.value)
@pn.depends(w_choose_channel.param.value)
def wcld_image_yt(yearr, channels):
    wordcloudt = WordCloud(width = 320, height = 550, background_color='#d3eaf2', font_path='C:\\Users\\USER\AppData\Local\Microsoft\Windows\Fonts\\Neue_soviet.ttf',\
        random_state=3, colormap='cool', color_func=lambda word, **kwargs:  '#6e1b29'if word=='россия' else "#524561", collocations=False, \
            relative_scaling=0.5).generate_from_frequencies(tag_df[(video_df['publishedAt']>=yearr[0])&(video_df['publishedAt']<=yearr[1])&(tag_df['channelName'].isin(channels))]['tag'].value_counts()[0:35])
    return wordcloudt.to_image()

# color_func=lambda word, **kwargs:  (255,0,0) if word=='россия' else colormap,
wordcloud = pn.bind(wcld_image_yt, w_yearrange, w_choose_channel)

pn.Column(w_yearrange, w_choose_channel, wordcloud)

BokehModel(combine_events=True, render_bundle={'docs_json': {'aef511b1-30ab-49a5-8f2a-50dc303b897c': {'defs': …

Column
    [0] DateRangeSlider(end=Timestamp('2023-02-15 1..., name='Date range', start=Timestamp('2009-03-18 2..., value=(Timestamp('2009-03-18 21:..., value_end=Timestamp('2023-02-15 1..., value_start=Timestamp('2009-03-18 2...)
    [1] CheckButtonGroup(button_type='primary', name='Channels', options=['БесогонTV', ...], orientation='vertical', value=['БесогонTV', ...])
    [2] ParamFunction(function, _pane=PNG)

Distribution

In [31]:
vid_hist = video_df[video_df['channelName']=='varlamov'][['channelName', 'likeCount']].hvplot.\
        hist('likeCount', by='channelName', alpha=0.5, bins=100, legend=False, width=500)

vid_hist

:NdOverlay   [channelName]
   :Histogram   [likeCount]   (likeCount_count)

<h3>Conclusion</h3>

Dashboard

In [32]:
@pn.depends(w_yearrange.param.value)
def year_mrkdn(yearr):
    return f'Channels from <u>{yearr[0].strftime("%B %Y")} to {yearr[1].strftime("%B %Y")}</u>'

# dashboard
template = pn.template.FastListTemplate(
    title="<b>YouTube channels analysis</b>",
    font_url='https://fonts.googleapis.com/css?family=Tangerine',
    sidebar=[pn.pane.Markdown('''<h3><b>Choose channels</b>:</h3>'''),
        w_choose_channel,
        pn.pane.Markdown('''<br><h3><b>Links</b>:</h3>
<b><a href="https://www.youtube.com/@Ekaterina_Schulmann">Ekaterina_Schulmann</a></b><br>
<b><a href="https://www.youtube.com/@varlamov">varlamov</a></b><br>
<b><a href="https://www.youtube.com/@Max_Katz">Max_Katz</a></b><br>
<b><a href="https://www.youtube.com/@besogontv">besogontv</a></b> (pro-Kremlin)<br>'''),
pn.pane.PNG('peace1.png', width=200)],
    sidebar_width=220,
    header_background='#d3eaf2',
    header_color='#4c3561',
    #accent_base_color='#daa520',
    main=[pn.Row(pn.Column(w_yearrange, wordcloud), 
        pn.Column(w_channel_feat, ch_barplot, w_video_feat, pn.Row(vid_barchrtr, vid_linechrt)))],
    main_max_width='100%', accent_base_color='#2a1340'
)

#template.show()
template.servable();